In [153]:
from datasets import load_dataset, Dataset
import pyarrow as pa

datasets = load_dataset("squad_kor_v1")
#datasets["train"]["question"]
#datasets

org_datasets = []
org_datasets.append(datasets["train"])
org_datasets.append(datasets["validation"])

new_pydict_list = []
for org_dataset in org_datasets:
    new_pydict = org_dataset.to_dict()
    #context_list = [func(data) for data in new_pydict["context"]]
    #new_pydict["context"] = context_list
    new_pydict_list.append(new_pydict)

new_table_list = []
for i in range(2):
    new_table_list.append(pa.Table.from_pydict(new_pydict_list[i]))

train_dataset = Dataset(new_table_list[0])
valid_dataset = Dataset(new_table_list[1])
#datasets = DatasetDict({"train": train_dataset, "validation": valid_dataset})
train_dataset

Reusing dataset squad_kor_v1 (/opt/ml/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/31982418accc53b059af090befa81e68880acc667ca5405d30ce6fa7910950a7)


Dataset({
    features: ['answers', 'context', 'id', 'question', 'title'],
    num_rows: 60407
})

In [159]:
import pandas as pd

datasets = load_dataset("squad_kor_v1")

df = pd.DataFrame(datasets["train"])
#Dataset.from_pandas(df)
Dataset(pa.Table.from_pandas(df))
#table = pa.Table.from_pandas(df

Reusing dataset squad_kor_v1 (/opt/ml/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/31982418accc53b059af090befa81e68880acc667ca5405d30ce6fa7910950a7)


Dataset({
    features: ['answers', 'context', 'id', 'question', 'title'],
    num_rows: 60407
})

In [161]:
from datasets import load_from_disk, Features, Sequence, Value
datasets = load_from_disk("/opt/ml/input/data/train_dataset")

train_dataset = datasets["train"]
eval_dataset = datasets["validation"]

def preprocess_features_of_Dataset(dataset):
    df = pd.DataFrame(dataset)

    features = Features({
        "answers": Sequence(feature={
            "text": Value(dtype="string", id=None),
            "answer_start": Value(dtype="int32", id=None)
        }, length=-1, id=None),
        "context": Value(dtype="string", id=None),
        "id": Value(dtype="string", id=None),
        "question": Value(dtype="string", id=None),
        "title": Value(dtype="string", id=None),
    })

    return Dataset.from_pandas(df, features=features)

preprocess_features_of_Dataset(train_dataset)

#train_dataset.map(features=features)

Dataset({
    features: ['answers', 'context', 'id', 'question', 'title'],
    num_rows: 3952
})

In [162]:
train_dataset.column_names

['title',
 'context',
 'question',
 'id',
 'answers',
 'document_id',
 '__index_level_0__']

# wiki docs

In [23]:
import json

def func(s):
    s = str(s)
    s = s.replace("\n\n", " ")
    s = s.replace("\n", " ")
    s = s.replace("#", "")
    s = s.replace("  ", " ")
    return s

data_path = "/opt/ml/input/data/wikipedia_documents.json"
with open(data_path) as f:
    dataset = json.load(f)
print(len(dataset.unique()))

cnt = 0
for k, v in dataset.items():
    dataset[k]["text"] = func(v["text"])

data_path = "/opt/ml/input/data/new_wikipedia_documents.json"
with open(data_path, "w") as f:
    json.dump(dataset, f)

In [89]:
dataset["0"]

{'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.',
 'corpus_source': '위키피디아',
 'url': 'TODO',
 'domain': None,
 'title': '나라 목록',
 'author': None,
 'html': None,
 'document_id': 0}

# datasets
## Load Dataset

In [6]:
from datasets import load_from_disk
from retrieval import get_dataset_with_retrieval
from transformers import TrainingArguments
training_args = TrainingArguments

data_path = "/opt/ml/input/data/train_dataset"
dataset = load_from_disk(data_path)
#dataset["validation"]["answers"]
#get_dataset_with_retrieval(dataset, training_args)

In [6]:
from datasets import load_from_disk

data_path2 = "/opt/ml/input/data/test_dataset"
dataset = load_from_disk(data_path2)
dataset

DatasetDict({
    validation: Dataset({
        features: ['id', 'question'],
        num_rows: 600
    })
})

In [7]:
for data in dataset["validation"]:
    if data["id"] == "mrc-0-003223":
        print(data["question"])

논문으로 플라톤주의의 허점을 언급한 사람은?


In [118]:
from datasets import load_from_disk

data_path = "/opt/ml/input/data/dummy_dataset"
dataset1 = load_from_disk(data_path)

data_path = "/opt/ml/input/data/train_dataset"
dataset2 = load_from_disk(data_path)

cnt = 0
for question in dataset1["train"]["question"]:
    if question not in dataset2["train"]["question"]\
    and question not in dataset2["validation"]["question"]:
        print(question)


각 측에서 주장된 포로교환 방법 중 선택된 것은?
한국에서 음식을 올려놓을때 쓰이는 세간의 이름은 무엇인가요?
미자르는 무엇의 일부인가?
해안포를 정착해 해안을 방어하기 위해 건설한 것은?
찰스 2세가 국내로 돌아오기 전에 있던 국가는?
나카무라 번이 메이지 정부에 굴복하게 된 결투의 이름은 무엇인가요?
예수의 테오토코스에 대해 부정적으로 인식했던 인물은?
윤치호가 사촌동생을 통해 만나려고 했던 인물은?
홉베마의 스승은 누구인가요?
슈타우펜베르크는 들 것에 실려나온 사람이 무엇을 덮고 있는 것을 보고 히틀러의 죽음을 단정했나?
아즈마의 초기 명칭은?
경찰은 어떤 기관의 GPS를 이용하여 오원춘을 체포할 수 있었나요?
자미토프가 개혁을 강행하기 위해 설립한 사병 집단은?
드레이어 감독이 잔 다르크에 비유한 사람은?
교전 당사자에 대한 전쟁 범죄 심판시 판단 기준이 되는 법은?
단공류가 일반 포유류와 다르다는 것을 알 수 있는 신체 부위는?
카쿠와 로브 루치의 현재 신분은?
출혈성 천연두 초기와 말기 환자 모두에게 증가하는 물질은?
사라스테의 후임은 누구인가?
미자가 일을 때려친 것은 누구 때문인가?
사이트에 올라오는 콘텐츠를 검열하는 주체는 누구인가?


In [86]:
from datasets import load_from_disk

data_path = "/opt/ml/input/data/train_dataset"
datasets = load_from_disk(data_path)
org_datasets = []
org_datasets.append(datasets["train"])
org_datasets.append(datasets["validation"])

org_datasets

[Dataset({
     features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
     num_rows: 3952
 }),
 Dataset({
     features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
     num_rows: 240
 })]

In [89]:
from datasets import concatenate_datasets

full_ds = concatenate_datasets([
    datasets["train"].flatten_indices(),
    datasets["validation"].flatten_indices(),
])
full_ds

Loading cached processed dataset at /opt/ml/input/data/train_dataset/train/cache-5864ec783ab77348.arrow
Loading cached processed dataset at /opt/ml/input/data/train_dataset/validation/cache-ec5c6075463087d0.arrow


Dataset({
    features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
    num_rows: 4192
})

In [141]:
n = 200
for data in org_datasets[0]:
    answer = data["answers"]["text"][0]
    if len(answer) > 30:
        print(data["question"], answer)
#org_datasets[0]["context"][0][235:237]

많은 갈등을 불러온 툴민의 책은? ≪인간의 이해: 개념의 집단적 사용 및 진화 (1972)≫
에릭 레이먼드의 자유 소프트웨어 철학을 대변하는 글은? 〈성당과 시장〉(The Cathedral and the Bazaar)
알바니아계 이탈리아 교회가 소속된 교구는? 룬그로 (이탈리아 대륙에 거주하는 아르버레셔인을 위한) 교구
판스워스 교수가 이전 배달원에 대해 한 대사는? "오 그 불쌍한 사람들... 그러나 그건 별로 중요하지 않아요! 중요한건 난 새로운 배달원이 필요했다는 거지!"
강중인이 경제사범을 비판하며 국민의 일대수치라 표현한 글은? 최근의 반도의 경제사범 ­국민의 신경제 윤리의 파악을 위하여­
도스토옙스키의 시력이 망가졌을 때 그는 어떤 방식으로 작품을 완성할 수 있었나? 도스토옙스키가 침대 누워 구술한 것을 아내 안나가 속기 하여
영어로 Presocratics란 뜻을 가진 낱말의 기원은? 독일의 고전 문헌학자 헤르만 딜스가 이들 철학자의 단편들을 한데 모아 엮은 책의 제목
두 개의 지역으로 줄어든 후에 킬리키아의 명칭은 무엇으로 변경되었나? 시리아킬리키아 포에니케 (Syria-Cilicia Phoenice)
윤치호가 일부 독립운동 지도자들로부터 비난 받았던 일기의 내용은? “나는 국경일에 일장기를 게양하는 것을 반대하지 않는다. 왜냐하면 우리가 일본의 통치하에 있는 한 우리는 그 통치의 명령에 복종해야 하기 때문이다.”
윤치호가 조지아주에서 재입학한 학교는? 에모리 대학(Oxford College of Emory University)
우룡은 그림에서 어떤 모습으로 묘사되었는가? 머리가 크고, 가늘고 허약해 보이는 동체를 꼬아놓은 모습
공공사업과 관련한 갈등이 유발되는 현장에서 사용되는 문구는 무엇인가? “나리타처럼 되지 말자(成田のようにならないようにしよう )”
작품 《트로일러스와 크리세이드》가 일으키는 논란을 해결하기 어렵다고 본 사람은? 프레드릭 S. 보아스 (Frederick S. Boas)
고고학자들은 어떤 상황일 때 원어를 활용해 원문을 복원하려

## context: remove "\\n\\n"
- 성능 향상을 보기 위해 context에 대해 처리, 저장, 실험한 후,  
  나중에 이를 불러와서 answer에 대해 처리합니다. 
- train_dataset에만 있는 데이터입니다.

In [48]:
from datasets import Dataset, DatasetDict

def func(s):
    s = str(s)
    s = s.replace("\\n\\n", " ")
    s = s.replace("\\n", " ")
    s = s.replace("#", "")
    s = s.replace("  ", " ")
    return s

new_pydict_list = []
for org_dataset in org_datasets:
    new_pydict = org_dataset.to_dict()
    context_list = [func(data) for data in new_pydict["context"]]
    new_pydict["context"] = context_list
    new_pydict_list.append(new_pydict)

new_table_list = []
for i in range(2):
    new_table_list.append(pa.Table.from_pydict(new_pydict_list[i]))

train_dataset = Dataset(new_table_list[0])
valid_dataset = Dataset(new_table_list[1])
train_datasets = DatasetDict({"train": train_dataset, "validation": valid_dataset})

In [50]:
train_datasets.save_to_disk(data_path + "_new")
train_datasets

## question: remove -요?

In [55]:
from datasets import load_from_disk

data_path1 = "/opt/ml/input/data/train_dataset"
dataset = load_from_disk(data_path1)
org_datasets = []
org_datasets.append(dataset["train"])
org_datasets.append(dataset["validation"])

data_path2 = "/opt/ml/input/data/test_dataset"
dataset = load_from_disk(data_path2)
org_datasets.append(dataset["validation"])
org_datasets

[Dataset({
     features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
     num_rows: 3952
 }),
 Dataset({
     features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
     num_rows: 240
 }),
 Dataset({
     features: ['id', 'question'],
     num_rows: 600
 })]

In [60]:
def func(s):
    s = str(s)
    if s[-2] == "요":
        s = s[:-2] + "?"
    if s[-2] == "나":
        s = s[:-2] + "는가?"
    return s

new_pydict_list = []
for org_dataset in org_datasets:
    new_pydict = org_dataset.to_dict()
    question_list = [func(data) for data in new_pydict["question"]]
    new_pydict["question"] = question_list
    new_pydict_list.append(new_pydict)

new_table_list = []
for new_pydict in new_pydict_list:
    new_table_list.append(pa.Table.from_pydict(new_pydict))

train_dataset = Dataset(new_table_list[0])
valid_dataset = Dataset(new_table_list[1])
train_datasets = DatasetDict({"train": train_dataset, "validation": valid_dataset})

test_dataset = DatasetDict({"validation": Dataset(new_table_list[2])})

In [62]:
train_datasets.save_to_disk(data_path1 + "_new")
test_dataset.save_to_disk(data_path2 + "_new")